# Data

In [1]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

--2025-11-30 20:53:12--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-30T21%3A39%3A16Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-30T20%3A38%3A26Z&ske=2025-11-30T21%3A39%3A16Z&sks=b&skv=2018-11-09&sig=KRE34x2PG%2BkId48ubACGsryOLjoWWrEwTBuN6MnTkd8%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDUzNzc5MiwibmJmIjoxNzY0NTM1OTkyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

In [2]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn

# Model

In [7]:
def make_model(input_size=200, learning_rate=0.002, num_classes=10):
    
    model = nn.Sequential(
        # Convolutional layer: 3 input channels, 32 output channels, 3x3 kernel
        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
        nn.ReLU(),
        
        # Max pooling: 2x2 pooling size
        nn.MaxPool2d(kernel_size=2, stride=2),
        
        # Flatten the output
        nn.Flatten(),
        
        # Fully connected layer with 64 neurons
        nn.Linear(32 * 99 * 99, 64),
        nn.ReLU(),
        
        # Output layer with num_classes neurons (one per hair type)
        nn.Linear(64, num_classes)
        # No activation here - we'll use CrossEntropyLoss which includes softmax
    )
    
    # Optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.8)
    
    # Loss function for multi-class classification
    criterion = nn.CrossEntropyLoss()
    
    return model, optimizer, criterion


def make_model(input_size=200, learning_rate = 0.01, size_inner=100, droprate=0.5):
    
    base_model = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=(3, input_size, input_size)
    )
    
    base_model.trainable=False
    
    #########################################
    
    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
    
    outputs = keras.layers.Dense(10)(drop)
    
    model = keras.Model(inputs, outputs)

    #########################################

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)
    
    model.compile(
        optimizer=optimizer, 
        loss=loss, 
        metrics=['accuracy']
    )

    return model


# Question 1

Which loss function you will use? CrossEntropyLoss()

In [8]:
!pip install torchsummary

In [9]:
model, optimizer, criterion = make_model()

In [12]:
# Option 1: Using torchsummary (install with: )
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same